In [32]:
import geopandas as gpd
import os
import pandas as pd
from scipy.io import netcdf
import netCDF4
ROOTFOLDER = "D:/Dropbox (Personal)/Personal Work/_Projects2023/01_city-never-was/_data"
RAW_PATH = "D:/Dropbox (Personal)/Personal Work/_commondata/_world_data/01_carbon_emission_ffdas/ffdas_flux_2013_2015.nc.gz/ffdas_flux_2015b.nc/ffdas_flux_2015.nc"
TRANSFORM_FOLDER = "D:/Dropbox (Personal)/Personal Work/_Projects2023/01_city-never-was/_data/_transformed/t_city_profiles"

In [2]:
nc = netCDF4.Dataset(RAW_PATH)
nc.variables.keys()

dict_keys(['latitude', 'latitude_edge', 'longitude', 'longitude_edge', 'flux'])

In [3]:
# convert the netcdf to a pandas dataframe
df = pd.DataFrame(nc.variables["flux"][:])
df["latitude"] = nc.variables["latitude"][:]
df.set_index('latitude', inplace=True)
df.columns = nc.variables["longitude"][:]
# df["longitude"] = nc.variables["longitude"][:]
df = df.stack().reset_index().rename(columns={0: 'flux','level_1':'longitude'})

In [4]:
RAW_FOLDER = f"{ROOTFOLDER}/_raw/_city_profiles"
df.to_csv(os.path.join(RAW_FOLDER, "r_ffdas_flux_2015.csv"), index=False)

## Aggregate to each city

In [8]:
# load all zone area for selected city
ROOTFOLDER = "D:/Dropbox (Personal)/Personal Work/_Projects2023/01_city-never-was/_data"
RAW_BOUND_FOLDER = f"{ROOTFOLDER}/_raw/r_boundary_osm"
TRANSFORM_FOLDER = f"{ROOTFOLDER}/_transformed/t_city_profiles/"
boundfiles = [f for f in os.listdir(RAW_BOUND_FOLDER) if f.endswith('.geojson')]
print("loading boundary files: ", len(boundfiles))

loading boundary files:  127


In [16]:
allbound = []
for f in boundfiles:
    temp = gpd.read_file(os.path.join(RAW_BOUND_FOLDER, f))
    temp['city_lower'] = f.split(".")[0]
    temp = temp[['city_lower','geometry']].to_crs("EPSG:4326")
    allbound.append(temp)
allbound = pd.concat(allbound).reset_index(drop = True)
allbound.head()
    

,city_lower,geometry
0,accra,"POLYGON ((-0.28413 5.57195, -0.28386 5.57090, ..."
1,amsterdam,"MULTIPOLYGON (((4.72878 52.40071, 4.75607 52.3..."
2,antwerp,"POLYGON ((4.21758 51.37389, 4.21826 51.37221, ..."
3,astrakhan,"POLYGON ((47.87112 46.26966, 47.87152 46.26632..."
4,athens,"MULTIPOLYGON (((22.89876 36.19485, 22.89888 36..."


In [14]:
# fix the data
# temp = gpd.read_file(os.path.join(RAW_BOUND_FOLDER, "athens.geojson"))
# temp.crs = "EPSG:2100"
# temp = temp.to_crs("EPSG:4326")
# temp.to_file(os.path.join(RAW_BOUND_FOLDER, "athens.geojson"), driver='GeoJSON')

In [18]:
allbound.to_file(os.path.join(TRANSFORM_FOLDER, "t_city_boundary.geojson"), driver='GeoJSON')

In [21]:
df = pd.read_csv(os.path.join(RAW_FOLDER, "r_ffdas_flux_2015.csv"))
df = df[df['flux']>0].reset_index(drop = True)
df = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.longitude, df.latitude), crs="EPSG:4326")
df

,latitude,longitude,flux,geometry
0,89.949997,33.750000,0.146428,POINT (33.75000 89.95000)
1,89.849998,-26.450001,0.283811,POINT (-26.45000 89.85000)
2,89.650002,37.849998,0.020916,POINT (37.85000 89.65000)
3,89.550003,-68.150002,0.054575,POINT (-68.15000 89.55000)
4,89.550003,-23.049999,0.064624,POINT (-23.05000 89.55000)
...,...,...,...,...
2257684,-77.750000,165.449997,0.006441,POINT (165.45000 -77.75000)
2257685,-77.750000,165.949997,0.006441,POINT (165.95000 -77.75000)
2257686,-77.750000,166.050003,0.006441,POINT (166.05000 -77.75000)
2257687,-77.750000,166.149994,0.006441,POINT (166.14999 -77.75000)


In [22]:
seldf = gpd.sjoin(df[['flux','geometry']], allbound, how="inner", op='intersects')
seldf.head()

c:\Users\yuanzfan\Anaconda3\envs\geo38\lib\site-packages\IPython\core\interactiveshell.py:3309: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):


,flux,geometry,index_right,city_lower
294779,1.472036,POINT (30.15000 60.05000),102,saintpetersburg
294780,5.579766,POINT (30.25000 60.05000),102,saintpetersburg
294781,7.247790,POINT (30.35000 60.05000),102,saintpetersburg
297275,10.190162,POINT (30.25000 59.95000),102,saintpetersburg
297276,21.470198,POINT (30.35000 59.95000),102,saintpetersburg


In [30]:
seldfagg = seldf.groupby('city_lower').agg({'flux':['sum', 'mean','count']}).reset_index().rename(
    columns = {'sum':'total_flux', 'mean':'mean_flux','count':'n_points'})

seldfagg.columns = seldfagg.columns.droplevel(0)
seldfagg.columns = ['city_lower', 'total_flux', 'mean_flux','flux_n_points']

In [33]:
seldfagg.sort_values("total_flux", ascending=False).to_csv(os.path.join(TRANSFORM_FOLDER, "t_ffdas_flux_2015.csv"), index=False)